In [1]:
import pyspark

In [2]:
pyspark.__file__

'/home/janiomjr/projects/docker-zoomcamp/data-engineering-zoomcamp/06-batch/code/.venv/lib/python3.13/site-packages/pyspark/__init__.py'

In [3]:
from pyspark.sql import SparkSession

In [10]:
spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/26 08:51:54 WARN Utils: Your hostname, MFdell, resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
26/02/26 08:51:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/26 08:51:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/26 08:51:56 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [11]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-11.parquet

--2026-02-26 08:52:24--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-11.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.165.4.125, 3.165.4.197, 3.165.4.195, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.165.4.125|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71134255 (68M) [binary/octet-stream]
Saving to: ‘yellow_tripdata_2025-11.parquet.1’

yellow_tripdata_202 100%[===================>]  67.84M  63.0MB/s    in 1.1s    

2026-02-26 08:52:25 (63.0 MB/s) - ‘yellow_tripdata_2025-11.parquet.1’ saved [71134255/71134255]



In [12]:
from pathlib import Path

# 1) Read November 2025 yellow parquet into Spark DF
df = spark.read.parquet("yellow_tripdata_2025-11.parquet")

print(df.count())
df.printSchema()

4181444
root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)



In [13]:
spark.version

'4.1.1'

In [14]:
# 2) Repartition to 4
df4 = df.repartition(4)

# 3) Write out to parquet folder (overwrite so you can rerun safely)
out_dir = Path("yellow_2025_11_parquet_4part")

# Spark writes to a DIRECTORY, not a single file
df4.write.mode("overwrite").parquet(str(out_dir))

print("Wrote to:", out_dir.resolve())


[Stage 6:>                                                          (0 + 4) / 4]

Wrote to: /home/janiomjr/projects/docker-zoomcamp/data-engineering-zoomcamp/06-batch/code/yellow_2025_11_parquet_4part


In [16]:
from pyspark.sql import functions as F

In [17]:
df \
    .withColumn('tpep_pickup_datetime', F.to_date(df.tpep_pickup_datetime)) \
    .filter("tpep_pickup_datetime = '2025-11-15'") \
    .count()

162604

In [22]:
from pyspark.sql import functions as F

df2 = (
    df
    .withColumn("pickup_date", F.to_date("tpep_pickup_datetime"))
    .withColumn(
        "duration_hours",
        (F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp("tpep_pickup_datetime")) / 3600.0
    )
)

(df2
 .groupBy("pickup_date")
 .agg(F.max("duration_hours").alias("max_duration_hours"))
 .orderBy(F.col("max_duration_hours").desc())
 .limit(5)
 .show(truncate=False)
)

[Stage 11:======================================>                  (8 + 4) / 12]

+-----------+------------------+
|pickup_date|max_duration_hours|
+-----------+------------------+
|2025-11-26 |90.64666666666666 |
|2025-11-27 |76.94833333333334 |
|2025-11-03 |76.21388888888889 |
|2025-11-07 |69.28861111111111 |
|2025-11-18 |67.08055555555555 |
+-----------+------------------+



In [23]:
!wget https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv

--2026-02-26 09:21:37--  https://d37ci6vzurychx.cloudfront.net/misc/taxi_zone_lookup.csv
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 3.165.4.195, 3.165.4.197, 3.165.4.125, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|3.165.4.195|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/csv]
Saving to: ‘taxi_zone_lookup.csv.1’

taxi_zone_lookup.cs 100%[===================>]  12.04K  --.-KB/s    in 0s      

2026-02-26 09:21:37 (412 MB/s) - ‘taxi_zone_lookup.csv.1’ saved [12331/12331]



In [24]:
!ls -lh taxi_zone_lookup.csv

-rw-r--r-- 1 janiomjr janiomjr 13K Feb 22  2024 taxi_zone_lookup.csv


In [25]:
zones = spark.read \
    .option("header", True) \
    .option("inferSchema", True) \
    .csv("taxi_zone_lookup.csv")

zones.printSchema()
zones.show(5)

root
 |-- LocationID: integer (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows


In [26]:
df.createOrReplaceTempView("yellow_trips")
zones.createOrReplaceTempView("zones")

spark.sql("""
SELECT z.Zone, COUNT(*) AS trips
FROM yellow_trips y
JOIN zones z
  ON y.PULocationID = z.LocationID
GROUP BY z.Zone
ORDER BY trips ASC
LIMIT 5
""").show(truncate=False)

[Stage 18:======================================>                  (8 + 4) / 12]

+---------------------------------------------+-----+
|Zone                                         |trips|
+---------------------------------------------+-----+
|Governor's Island/Ellis Island/Liberty Island|1    |
|Eltingville/Annadale/Prince's Bay            |1    |
|Arden Heights                                |1    |
|Port Richmond                                |3    |
|Rikers Island                                |4    |
+---------------------------------------------+-----+

